# 13/04/2020: Analysis of Covid-19

In [143]:
import os
import pandas as pd
import glob
import datetime
import plotly
import plotly.graph_objs as go

#### This analysis is possible thanks to the daily reports available here: https://github.com/CSSEGISandData/COVID-19

In [10]:
path = r'./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports'

if os.path.exists(path):
    file_list = glob.glob(path+'/*.csv')
else:
    print(path+' does not exist')

In [13]:
file_list[:5]

['./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports\\01-22-2020.csv',
 './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports\\01-23-2020.csv',
 './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports\\01-24-2020.csv',
 './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports\\01-25-2020.csv',
 './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports\\01-26-2020.csv']

### First we have to map the cols and country names 

In [111]:
col_mapping = {
    'Last Update': 'Last_Update',
    'Long_': 'Longitude',
    'Lat': 'Latitude',
    'Country/Region': 'Country_Region'  
}

country_mapping = {
    'Mainland China':'China',
    'Russia': 'Russian Federation',
    'South Korea': 'Korea, Republic of',
    'Korea, South': 'Korea, Republic of',
    'Republic of Korea': 'Korea, Republic of',
    'Taiwan': 'Taiwan, Province of China',
    'Taiwan*': 'Taiwan, Province of China',
    'UK': 'United Kingdom',
    'Tanzania': 'Tanzania, United Republic of',
    'The Bahamas': 'Bahamas',
    'Bahamas, The': 'Bahamas',
    'US': 'United States',
    'Venezuela': 'Venezuela, Bolivarian Republic of',
    'Bolivia': 'Bolivia, Plurinational State of',
    'Vietnam': 'Viet Nam',
    'Palestine': 'Palestine, State of',
    'West Bank and Gaza': 'Palestine, State of',
    'Republic of Moldova': 'Moldova, Republic of',
    'Moldova': 'Moldova, Republic of',
    'Macedonia': 'Macedonia, the Former Yugoslav Republic of',
    'North Macedonia': 'Macedonia, the Former Yugoslav Republic of',
    'Macau': 'Macao',
    'Laos': "Lao People's Democratic Republic",
    'Ivory Coast': 'Côte d’Ivoire',
    "Cote d'Ivoire": 'Côte d’Ivoire',
    'Iran': 'Iran, Islamic Republic of',
    'Eswatini': 'Swaziland',
    'Czechia': 'Czech Republic',
    'Congo (Kinshasa)': 'Congo, the Democratic Republic of the',
    'Congo (Brazzaville)': 'Congo',
    'Cabo Verde': 'Cape Verde',
    'Burma': 'Myanmar',
    'Brunei': 'Brunei Darussalam',
    'Holy See': 'Holy See (Vatican City State)',
    'Syria': 'Syrian Arab Republic'
}

cols_to_drop = ['Admin2', 'FIPS', 'ISO3', 'Incident_Rate', 'Combined_Key', 'People_Hospitalized', 'People_Tested', 'Province_State', 'UID']

### Then we'll transform each report file into a dataframe and concat them

In [129]:
dfs = []
for f in file_list:
    data = pd.read_csv(f)
    for col in cols_to_drop:
        if col in data.columns:
            data.drop(columns=col, inplace=True)
    for key, value in col_mapping.items():
        if key in data.columns:
            data.rename(columns={key: value}, inplace=True)
    dfs.append(data)
    
df = pd.concat(dfs, sort=True)
df.fillna(0, inplace=True)
df = df[df['Country_Region'] != 'Namibia']
df['Country_Region'].replace(country_mapping, inplace=True)

In [130]:
df.head()

,Active,Confirmed,Country_Region,Deaths,Last_Update,Latitude,Longitude,Province/State,Recovered
0,0.0,1.0,China,0.0,1/22/2020 17:00,0.0,0.0,Anhui,0.0
1,0.0,14.0,China,0.0,1/22/2020 17:00,0.0,0.0,Beijing,0.0
2,0.0,6.0,China,0.0,1/22/2020 17:00,0.0,0.0,Chongqing,0.0
3,0.0,1.0,China,0.0,1/22/2020 17:00,0.0,0.0,Fujian,0.0
4,0.0,0.0,China,0.0,1/22/2020 17:00,0.0,0.0,Gansu,0.0


### And that's our dataframe. Now where ready to make some changes

In [131]:
#Transform the 'Last_Update' column to datetime and create the date column using only the date part

df['Last_Update'] = pd.to_datetime(df['Last_Update'])
df['date'] = df['Last_Update'].dt.date
df.head()

,Active,Confirmed,Country_Region,Deaths,Last_Update,Latitude,Longitude,Province/State,Recovered,date
0,0.0,1.0,China,0.0,2020-01-22 17:00:00,0.0,0.0,Anhui,0.0,2020-01-22
1,0.0,14.0,China,0.0,2020-01-22 17:00:00,0.0,0.0,Beijing,0.0,2020-01-22
2,0.0,6.0,China,0.0,2020-01-22 17:00:00,0.0,0.0,Chongqing,0.0,2020-01-22
3,0.0,1.0,China,0.0,2020-01-22 17:00:00,0.0,0.0,Fujian,0.0,2020-01-22
4,0.0,0.0,China,0.0,2020-01-22 17:00:00,0.0,0.0,Gansu,0.0,2020-01-22


In [132]:
#Group by date and country to check the sum of each information (confirmed cases, deaths ...)

df_by_date_country = df.groupby(['Country_Region', 'date']).sum()
df_by_date_country.reset_index(inplace=True)
df_by_date_country.drop(columns=['Latitude', 'Longitude'], inplace=True)
df_by_date_country.set_index('date', inplace=True)
df_by_date_country.head()

,Country_Region,Active,Confirmed,Deaths,Recovered
date,,,,,
2020-02-28,Azerbaijan,0.0,1.0,0.0,0.0
2020-02-24,Afghanistan,0.0,13.0,0.0,0.0
2020-03-08,Afghanistan,0.0,8.0,0.0,0.0
2020-03-10,Afghanistan,0.0,5.0,0.0,0.0
2020-03-11,Afghanistan,0.0,21.0,0.0,0.0


In [133]:
#The iso_3 file contains the iso-3 code for each country

iso_3 = pd.read_csv('./resources/iso-3.csv')
iso_3.head()

,Name,Code
0,Afghanistan,AF
1,Ã…land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [145]:
df_by_date_country_iso = df_by_date_country.merge(iso_3, how='left', left_on='Country_Region', right_on='Name')
df_by_date_country_iso.drop('Name', axis=1, inplace=True)
df_by_date_country_iso.dropna(subset=['Code'], inplace=True)
df_by_date_country_iso.head()

,Country_Region,Active,Confirmed,Deaths,Recovered,Code
1,Afghanistan,0.0,13.0,0.0,0.0,AF
2,Afghanistan,0.0,8.0,0.0,0.0,AF
3,Afghanistan,0.0,5.0,0.0,0.0,AF
4,Afghanistan,0.0,21.0,0.0,0.0,AF
5,Afghanistan,0.0,11.0,0.0,0.0,AF


In [109]:
df_france = df_by_date_country[df_by_date_country['Country_Region'] == 'France']
df_us = df_by_date_country[df_by_date_country['Country_Region'] == 'United States']
df_spain = df_by_date_country[df_by_date_country['Country_Region'] == 'Spain']
df_italy = df_by_date_country[df_by_date_country['Country_Region'] == 'Italy']
df_br = df_by_date_country[df_by_date_country['Country_Region'] == 'Brazil']

### Timeseries: Next we're going to plot our first graph based on the countries we chose above and compare them

In [87]:
data = [
        go.Scatter(
            x=df_france.index,
            y=df_france['Confirmed'],
            name='France'    
        ),
        go.Scatter(
            x=df_us.index,
            y=df_us['Confirmed'],
            name='US'
        ),
        go.Scatter(
            x=df_spain.index,
            y=df_spain['Confirmed'],
            name='Spain'
        ),
        go.Scatter(
            x=df_italy.index,
            y=df_italy['Confirmed'],
            name='Italy'
        ),
        go.Scatter(
            x=df_br.index,
            y=df_br['Confirmed'],
            name='Brazil'
        )
    ]

layout = go.Layout(
        title='Confirmed Cases Per Country'
    )

plotly.offline.plot({
        "data": data,
        "layout": layout,
    },auto_open=True,filename='output/cases_comparison_scatter_plot.html')

'output/cases_comparison_scatter_plot.html'

### This next plot shows the number of confirmed cases in each country

In [157]:
import plotly.express as px
fig = px.scatter_geo(df_by_date_country_iso, locations="Country_Region",
                     color="Country_Region", # which column to use to set the color of markers
                     hover_name="Country_Region", # column added to hover information
                     size=df_by_date_country_iso["Confirmed"]*10, # size of markers
                     locationmode='country names',
                     text = df_by_date_country_iso['Country_Region'],
                     projection="natural earth")
fig.show()